# MD2SL - Master in Data Science and Statistical Learning

**Numerical Calculus and Linear Algebra**

Exercises 02: Linear systems: LU decomposition and partial pivoting

Deadline: 31/03/2023 

In [67]:
# Installing packages
import numpy as np
import math
from scipy import *
import scipy.linalg as la
import matplotlib.pyplot as plt

# Exercise 1
Let $n\in\mathbb{N}$ and $\widehat{\mathbf{x}} = \pmatrix{1.1 \\ \vdots \\ 1.1}\in\mathbb{R}^n$.

1. Consider the matrix $A = (a_{ij})\in\mathbb{R}^{n\times n}$, so that
\begin{equation}
  a_{ij} = \cos(j\theta),\ \text{with}\ \theta = \frac{(2i+1)\pi}{2n}
\end{equation}

  and the linear system $A\cdot\mathbf{x} = \mathbf{b}$, with $\mathbf{b} = A\cdot\widehat{\mathbf{x}}$.

  For $n=5,10,15,20,25,30$, solve the system using the functions `solveLU(A,b)` if possible, otherwise use `solveLUP(A,b)`.
  Print the following values, depending on $n$
  -  conditioning of the matrix
  -  euclidean norm of the relative error
  -  euclidean norm of the residual error

  and comment the results.


Exercise 01 - Solution

In [68]:
# Matrix A
def get_matrix_ex1(n):
    
    # TO DO
    A = np.zeros((n, n), dtype = float)
    
    for i in range(n):
        
        for j in range(n):
            
            A[i,j] = float(math.cos(float(j) * (math.pi * (1. + 2. * float(i))) / (2. * float(n))))
            
    return A

# A = get_matrix_ex1(int(10))
# print(A)    


def upperTriangular(A,b):
    
    # A: upper triangular coefficient matrix
    # b: vector of costant terms
    
    x = b.copy()

    for j in range(A.shape[1]-1, -1, -1):  # n-2:-1:0
        
        x[j] = x[j]/A[j,j]
        
        for i in range(j):
            
            x[i] = x[i] - A[i,j] * x[j]
            
    return x


def gaussLU(A):
    
    #Get the number of rows
    n = A.shape[0]
    LU = A.copy()
    
    #Loop over rows
    for k in range(n-1):
        
        if LU[k,k] == 0:
            
            raise ValueError('A has no lu decomposition.')
            
        LU[k+1:,k]     = LU[k+1:,k]/LU[k,k] # calcolo moltiplicatori
        
        LU[k+1:,k+1:] = LU[k+1:,k+1:]- np.outer(LU[k+1:,k],LU[k,k+1:]) # outer vector product
    
    return LU


def lowerTriangularUnitDiagonal(A,b):
    
    assert A.shape[0] == A.shape[1]  # m == n, otherwise break
    
    x = b.copy()
    
    for j in range(A.shape[1]):
        
        for i in range(j+1, A.shape[1]):
            
            x[i] = x[i]-A[i,j]*x[j]
            
    return x


def solveLU(LU, b):
    
    assert LU.shape[0] == LU.shape[1]  # m == n, otherwise break
    
    n = LU.shape[0]
    
    assert b.shape[0] == n             # b length == n according to LU, otherwise break
    
    x = lowerTriangularUnitDiagonal(LU,b)
    x = upperTriangular(LU,x)
    
    rnorm = np.linalg.norm(b-(np.eye(n) + np.tril(LU,-1)) @ np.triu(LU) @ x, 2) # matrix multiplication with @ 
    
    return x, rnorm


def gaussLUP(A):
    
    LU = A.copy()
    n = LU.shape[0]
    piv = np.arange(0,n)
    
    for k in range(n-1):
        
        # pivoting
        r_idx = np.argmax(abs(LU[k:,k])) + k
        
        if LU[r_idx,k]==0:
            raise ValueError('Singular matrix.')
        
        piv[[k,r_idx]] = piv[[r_idx,k]]
        LU[[k,r_idx]] = LU[[r_idx,k]]
        
        # LU 
        LU[k+1:,k]     = LU[k+1:,k]/LU[k,k]
        LU[k+1:,k+1:] = LU[k+1:,k+1:]- np.outer(LU[k+1:,k],LU[k,k+1:])  # outer vector product
        
    return LU, piv


def solveLUP(LU,b,P):
    
    bp = b[P]  # otherwise also b is changed
    x, rnorm = solveLU(LU,bp)
    
    return x, rnorm


In [69]:
print('\n--------------------------------------------------------')
print('n      cd       res     res-pivot      err     err-pivot')
print('--------------------------------------------------------')


for n in range(5, 35, 5): # range(start, stop, step) -> [5, 10, 15, 20, 25, 30]
    
    A = get_matrix_ex1(n)
    
    # exact solution of the system
    x_hat = np.array(n * [1.1])
    
    # known vector
    b = A @ x_hat
    
    LU       = gaussLU(A)
    x, rnorm = solveLU(LU, b)
    err_r    = np.linalg.norm(x - x_hat) / np.linalg.norm(x)  # relative error

    LU_piv, P        = gaussLUP(A)
    x_piv, rnorm_piv = solveLUP(LU_piv, b, P)
    err_r_piv        = np.linalg.norm(x_piv - x_hat) / np.linalg.norm(x_piv)  # retive error
    
    print(f'{n:2}  {np.linalg.cond(A):.2e}  {rnorm:.2e}   {rnorm_piv:.2e}   {err_r:.2e}   {err_r_piv:.2e}')
    
print('--------------------------------------------------------\n')



--------------------------------------------------------
n      cd       res     res-pivot      err     err-pivot
--------------------------------------------------------
 5  1.41e+00  2.33e-15   8.60e-16   3.38e-16   2.21e-16
10  1.41e+00  1.55e-12   2.82e-15   1.28e-13   5.49e-16
15  1.41e+00  4.11e-10   6.77e-15   6.64e-11   3.97e-16
20  1.41e+00  5.71e-07   1.87e-14   1.23e-08   9.14e-16
25  1.41e+00  1.66e-04   2.02e-14   5.23e-06   9.05e-16
30  1.41e+00  2.04e-02   1.65e-14   1.14e-03   7.33e-16
--------------------------------------------------------



- What happens to the condition number of $A$? What does it mean?

The conditional number is steady. This means that the conditioning of the problem doesn't change with the dimension of the matrix A. 

Moreover, 1.41 is a good condition number, so the matrix is well-conditioned.

  
- How do the naive and pivot residuals behave?

The naive residuals grow rapidly (in the order of 10^2÷3 every 5n)  with the increasing of n. 

Similarly, the pivot residuals grow but at a slower pace than before.

  
- How do the naive and pivot relative errors behave?

The naive relative errors grow fast, while the pivot errors are pretty steady and small.


---

Please, note that some outcomes in Jupyter where displayed differently than in Google Colab.


# Exercise 2
Let $n\in\mathbb{N}$ and $\widehat{\mathbf{x}} = \pmatrix{1.1 \\ \vdots \\ 1.1}\in\mathbb{R}^n$.

1. Consider the matrix $A = (a_{ij})\in\mathbb{R}^{n\times n}$, so that
\begin{equation}
  a_{ij} = (i+1)^{j}
\end{equation}

  and the linear system $A\cdot\mathbf{x} = \mathbf{b}$, with $\mathbf{b} = A\cdot\widehat{\mathbf{x}}$.

  For $n=1,\dots,10$, solve the system using the functions `solveLU(A,b)` if possible, otherwise use `solveLUP(A,b)`.
  Print following values, depending on $n$
  -  conditioning of the matrix
  -  euclidean norm of the relative error
  -  euclidean norm of the residual error

  and comment the results.
  
  


Exercise 02 - Solution

In [70]:
def get_matrix_ex2(n):
    # TO DO
    A = np.zeros((n, n), dtype = int)
    
    for i in range(n):
        
        for j in range(n):
            
            A[i,j] = (i + 1) ** j
            
    return A

# A = get_matrix_ex2(10)
# print(A)   

In [71]:
print('-------------------------------------------------------')
print('n     cd        res     res-pivot     err     err-pivot')
print('-------------------------------------------------------')

for n in range(1, 11): # range(start, stop, step) -> [5, 10, 15, 20, 25, 30]
    
    A = get_matrix_ex2(n)
    
    # exact solution of the system
    x_hat = np.array(n * [1.1])
    
    # known vector
    b = A @ x_hat
    
    LU       = gaussLU(A)
    x, rnorm = solveLU(LU, b)
    err_r    = np.linalg.norm(x - x_hat) / np.linalg.norm(x)  # relative error

    LU_piv, P        = gaussLUP(A)
    x_piv, rnorm_piv = solveLUP(LU_piv, b, P)
    err_r_piv        = np.linalg.norm(x_piv - x_hat) / np.linalg.norm(x_piv)  # retive error
    
    print(f'{n:2}  {np.linalg.cond(A):.2e}  {rnorm:.2e}   {rnorm_piv:.2e}   {err_r:.2e}   {err_r_piv:.2e}')

print('--------------------------------------------------------\n')


-------------------------------------------------------
n     cd        res     res-pivot     err     err-pivot
-------------------------------------------------------
 1  1.00e+00  0.00e+00   0.00e+00   0.00e+00   0.00e+00
 2  6.85e+00  0.00e+00   0.00e+00   0.00e+00   0.00e+00
 3  7.09e+01  0.00e+00   0.00e+00   8.72e-16   7.00e-01
 4  1.17e+03  0.00e+00   7.11e-15   7.96e-15   9.57e-01
 5  2.62e+04  1.30e-13   2.27e-13   5.21e-14   9.99e-01
 6  7.31e+05  9.39e-13   9.39e-13   4.85e-12   1.00e+00
 7  2.45e+07  8.19e-12   3.26e-11   1.50e-10   1.00e+00
 8  9.52e+08  9.33e-10   1.17e-10   1.16e-09   1.00e+00
 9  4.23e+10  3.84e-09   7.51e-09   2.28e-08   1.00e+00
10  2.11e+12  5.96e-08   6.74e-08   2.44e-06   1.00e+00
--------------------------------------------------------



- What happens to the condition number of $A$? What does it mean?

Differently than the previous case, in this problem the conditional number of A grows with n, reaching high values, so the matrix is ill-conditioned.

- How do the naive and pivot residuals behave?

The naive residuals grow rapidly with n, starting from n = 5.

Similarly, the pivot residuals grow but faster than before, starting from n = 4.

- How do the naive and pivot relative errors behave

The naive relative errors is zero until n = 3, when it starts to grow.

While the pivot errors are pretty steady.

---

Please, note that some outcomes in Jupyter where displayed differently than in Google Colab.


# Exercise 3
Let $A\in\mathbb{R}^{n\times n}$ a tridiagonal matrix
\begin{equation}
  A = \pmatrix{d_0     & a_0    &        &         & \huge 0 \\
               c_1     & \ddots & \ddots &         &         \\
                       & \ddots & \ddots & \ddots  &         \\
                       &        & \ddots & \ddots  & a_{n-2} \\
               \huge 0 &        &        & c_{n-1} & d_{n-1} \\}
\end{equation}

for which its LU decomposition exists. Hence, $A = L\cdot U$ with
\begin{equation}
L = \pmatrix{1       &        &         & \huge0 \\
             l_1     & \ddots &         &        \\
                     & \ddots & \ddots  &        \\
             \huge 0 &        & l_{n-1} & 1},
\quad
U = \pmatrix{u_0       &  a_0   &         & \huge0 \\
                     & \ddots & \ddots  &        \\
                     &        & \ddots  & a_{n-2}\\
             \huge 0 &        &         & u_{n-1}}.
\end{equation}


Example given for $n=4$.
\begin{equation}
\pmatrix{d_0 & a_0 & 0   & 0   \\
         c_1 & d_1 & a_1 & 0   \\
         0   & c_2 & d_2 & a_2 \\
         0   & 0   & c_3 & d_3} = \pmatrix{1   & 0   & 0   & 0 \\
                                           l_2 & 1   & 0   & 0 \\
                                           0   & l_3 & 1   & 0 \\
                                           0   & 0   & l_4 & 1 }\pmatrix{u_0   & a_0 & 0   & 0   \\
 0     & u_1 & a_1 & 0   \\
 0     & 0   & u_2 & a_2 \\
 0     & 0   & 0   & u_3  }
\end{equation}


In particular,
\begin{array}{clll}
          & c_1 = l_1u_0        &  c_2 = l_2u_1       & c_3 = l_3u_2\\
d_0 = u_0 & d_1 = l_1a_0 + u_1  &  d_2 = l_2a_1 + u_2 & d_3 = l_3a_2 + u_3.\\
\end{array}



1. Write a python function `thomas(c,d,a)` which takes in input the diagonals   of a tridiagonal square matrix $A\in\mathbb{R}^{n\times n}$
  - $c:$  lower diagonal
  - $d:$  main diagonal
  - $a:$  upper diagonal
  
  and returns the diagonals of the lu decomposition $A = L\cdot U$
  - $l:$  lower diagonal of $L$
  - $u:$  upper diagonal of $U$ 

  by implementing the following *Thomas algorithm*.

  **Input:** $c$, $d$, $a$
  1. $u_0$ = $d_0$
  2. for $i=1,\dots,n-1$ 
    - $l_i$ = $c_i / u_{i-1}$
    - $u_i$ = $d_i - l_ia_{i-1}$
  **Output:** $l$, $u$

2. Test the function `thomas(c, d, a)` on the following tridiagonal matrices.

\begin{equation}
\begin{split}
(a)\ \pmatrix{1 & 4 & 0 & 0\\
             3 & 4 & 1 & 0\\
             0 & 2 & 3 & 4\\
             0 & 0 & 1 & 3}
             \\
             \\
(b)\ \pmatrix{2  & 1  & 0  & 0\\
             -1 & 2  & 1  & 0\\
             0  & -1 & 2  & 1\\
             0  & 0  & -1 & 2 
            }
            \\
            \\
(c)\ \pmatrix{ 2 &  1 &  0 &  0 &  0 & 0\\
              -1 &  4 &  1 &  0 &  0 & 0\\
               0 & -1 &  4 &  1 &  0 & 0\\
               0 &  0 & -1 &  4 &  1 & 0\\
               0 &  0 &  0 & -1 &  4 & 1\\
               0 &  0 &  0 &  0 & -1 & 2}
\end{split}
\end{equation}
  - reconstruct the $L$ and $U$ matricies of their LU decomposition;
  - compute the recontruction error as $\texttt{norm}(A - L\cdot U)$.

Exercise 3.1 - Solution

In [72]:
def thomas(c, d, a):
    
    # TO DO
    n = len(d)
    u = np.zeros(n)
    l = np.zeros(n - 1)
    u[0] = d[0]
    
    for i in range(n - 1):
        
        l[i] = c[i] / u[i]
        
        u[i+1] = d[i+1] - l[i] * a[i]
        
    return l, u

Exercise 3.1.a - Solution

In [73]:
# fill in
A =  np.array([[1, 4, 0, 0],
               [3, 4, 1, 0],
               [0, 2, 3, 4],
               [0, 0, 1, 3]])

d = np.diag(A)      # main diagonal of A
c = np.diag(A, -1)  # lower diagonal of A
a = np.diag(A, +1)  # upper diagonal of A

l, u = thomas(c, d, a)

L = np.diag([1] * A.shape[0]) + np.diag(l, -1)  # assemble

U = np.diag(u) + np.diag(a, +1)                 # assemble

error = np.linalg.norm(A - L @ U)

print(f'Reconstruction error: {error}\n\n{(L @ U)}')


Reconstruction error: 0.0

[[1. 4. 0. 0.]
 [3. 4. 1. 0.]
 [0. 2. 3. 4.]
 [0. 0. 1. 3.]]


Exercise 3.1.b - Solution

In [74]:
# fill in
A =  np.array([[ 2,  1,  0, 0],
               [-1,  2,  1, 0],
               [ 0, -1,  2, 1],
               [ 0,  0, -1, 2]])

d = np.diag(A)      # main diagonal of A
c = np.diag(A, -1)  # lower diagonal of A
a = np.diag(A, +1)  # upper diagonal of A

l, u = thomas(c, d, a)

L = np.diag([1] * A.shape[0]) + np.diag(l, -1)  # assemble

U = np.diag(u) + np.diag(a, +1)                 # assemble

error = np.linalg.norm(A - L @ U)

print(f'Reconstruction error: {error}\n\n{(L @ U)}')


Reconstruction error: 2.220446049250313e-16

[[ 2.  1.  0.  0.]
 [-1.  2.  1.  0.]
 [ 0. -1.  2.  1.]
 [ 0.  0. -1.  2.]]


Exercise 3.1.c - Solution

In [75]:
# fill in
A =  np.array([[ 2,  1,  0,  0,  0,  0],
               [-1,  4,  1,  0,  0,  0],
               [ 0, -1,  4,  1,  0,  0],
               [ 0,  0, -1,  4,  1,  0],
               [ 0,  0,  0, -1,  4,  1],
               [ 0,  0,  0,  0, -1,  2]])

d = np.diag(A)      # main diagonal of A
c = np.diag(A, -1)  # lower diagonal of A
a = np.diag(A, +1)  # upper diagonal of A

l, u = thomas(c, d, a)

L = np.diag([1] * A.shape[0]) + np.diag(l, -1)  # assemble

U = np.diag(u) + np.diag(a, +1)                 # assemble

error = np.linalg.norm(A - L @ U)

print(f'Reconstruction error: {error}\n\n{(L @ U)}')


Reconstruction error: 6.280369834735101e-16

[[ 2.  1.  0.  0.  0.  0.]
 [-1.  4.  1.  0.  0.  0.]
 [ 0. -1.  4.  1.  0.  0.]
 [ 0.  0. -1.  4.  1.  0.]
 [ 0.  0.  0. -1.  4.  1.]
 [ 0.  0.  0.  0. -1.  2.]]
